In [39]:
#below imports everthing we need
import pandas as pd
import numpy as np
import requests
import warnings
import sys
import json


mlbsong=pd.read_csv('MLB_Walkup.csv')
mlbplayer=input("Enter an active MLB player or type 'quit': ")

songdata = mlbsong
songdf = pd.DataFrame(songdata)

while True:
    try:
        #if user enters 'quit', end program
        if mlbplayer == 'quit':
            break        
        
        #makes dataframe for team, player, artist, song, playlist link
        row =(songdf.loc[songdf['Player'] == mlbplayer])
        row = pd.DataFrame({'Team': row['Team'],'Player': row['Player'], 'Artist': row['Artist'], 'Song': row['Song'], 'Playlist': row['Playlist']})
        #row

        #makes playlist link clickable
        def make_clickable(val):
            return '<a target="_blank" href="{}">{}</a>'.format(val, val)

        row.style.format({'Playlist': make_clickable})

        if songdf['Player'].count == 0:
            print("player doesn't exist")
        else:

            #beginning of mlb api
            url=("http://lookup-service-prod.mlb.com//json/named.search_player_all.bam?sport_code='mlb'&active_sw='Y'&name_part='%s'" % (mlbplayer)) 
            response=requests.get(url)
            response.ok

            #below gets player id for url2 and url3 as well as position to determine whether player is batter or pitcher
            player=response.json()
            playerid=player['search_player_all']['queryResults']['row']['player_id']
            position=player['search_player_all']['queryResults']['row']['position']
            playername=player['search_player_all']['queryResults']['row']['name_display_first_last']

            if position == "P":
                #below is for pitchers
                purl=("http://lookup-service-prod.mlb.com//json/named.sport_pitching_tm.bam?league_list_id='mlb'&game_type='R'&season='2018'&player_id='%s'" % (playerid))
                response3=requests.get(purl)
                response3.ok
                p_stats=response3.json()
                wins=p_stats['sport_pitching_tm']['queryResults']['row']['w']
                innings=p_stats['sport_pitching_tm']['queryResults']['row']['ip']
                strikeouts=p_stats['sport_pitching_tm']['queryResults']['row']['so']
               # print("Player", playername, "Position", position, "Wins:", wins, "Innings:", innings, "Strikeouts:", strikeouts)
                #print(row)
                #picture()
                pitchers = pd.DataFrame({'Team': row['Team'],'Player': row['Player'], 'Artist': row['Artist'], 'Song': row['Song'], 'Playlist': row['Playlist'],'Wins': wins, 'Innings': innings, 'Strikeouts': strikeouts})
                picture()
                print(pitchers)
                break
                
                
                
            
            else:
                #below is for batters
                burl=("http://lookup-service-prod.mlb.com//json/named.sport_hitting_tm.bam?league_list_id='mlb'&game_type='R'&season='2018'&player_id='%s'" % (playerid))
                response2=requests.get(burl)
                response2.ok
                b_stats=response2.json()
                average=b_stats['sport_hitting_tm']['queryResults']['row']['avg']
                hr=b_stats['sport_hitting_tm']['queryResults']['row']['hr']
                rbi=b_stats['sport_hitting_tm']['queryResults']['row']['rbi']
                #print("Player", playername, "Position", position, "Average:", average, "Home Runs:", hr, "RBI:", rbi)
                #print(row)
                #picture()
                batters = pd.DataFrame({'Team': row['Team'],'Player': row['Player'], 'Artist': row['Artist'], 'Song': row['Song'], 'Playlist': row['Playlist'], 'Average': average, 'Home Runs': hr, 'RBI': rbi})
                picture()
                print(batters)
                break
                
                
            
            
    except KeyError:
        print("This player does not exist, is spelled wrong, was not active in 2018, or is not in the database of mlb.com/entertainment")
        break


Enter an active MLB player or type 'quit': Albert Pujols


       Team         Player  Artist                   Song  \
644  Angels  Albert Pujols  Lecrae  Don’t Waste Your Life   

                                              Playlist Average Home Runs RBI  
644  https://open.spotify.com/user/crfv73qfpinw7aq9...    .245        19  64  


In [32]:
from IPython.display import display, HTML
def picture():
    display (HTML("<img src='https://content.mlb.com/images/headshots/current/60x60/%s@3x.png'>"%(playerid)))

In [41]:
#we need to make batter dataframe (bdf) and pitcher dataframe (pdf) everthing else below is how we are going to merge it, I think

bmerged_dataframe = songdf.merge(bdf, left_on='playername', right_on='playername', how='inner')
bmerged_dataframe.head()

pmerged_dataframe = songdf.merge(pdf, left_on='playername', right_on='playername', how='inner')
pmerged_dataframe.head()

NameError: name 'bdf' is not defined

In [ ]:
bdg